# Assignment 1 part 2

Authors: Akos Engelmann, Gergely Parady, Ipek Cakin, Simon Jasansky

## Preprocessing of data

First, we build two separate dataframes: one with the unique artists, and one with the unique genres. 
Additionally, we add columns for the IRI endings, where we replace spaces with underscores. 

In [2]:
import pandas as pd
import numpy as np
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import XSD, RDF, RDFS
from rdflib import Graph
from urllib.parse import quote

In [ ]:
music = pd.read_csv("data/music.csv")

# extract unique artists and genres
artists = music.artist.unique()
genre = music["top genre"].unique()

# generate dataframes again
artists_df = pd.DataFrame({"artist": artists, "artist_iri": artists})
genres_df = pd.DataFrame({"top genre":  genre, "top genre_iri":  genre})

# replace white spaces with underscores
artists_df["artist_iri"].replace(" ", "_", regex = True, inplace = True)
genres_df["top genre_iri"].replace(" ", "_", regex = True, inplace = True)
# we also have to do that in the music dataframe
music["artist_iri"] = music["artist"].replace(" ", "_", regex = True)
music["top genre_iri"] = music["top genre"].replace(" ", "_", regex = True)

# export
artists_df.to_csv("intermediate/artists.csv", index_label="index")
genres_df.to_csv("intermediate/genres.csv", index_label="index")
music.to_csv("intermediate/music.csv", index=False)

In [10]:
g = Graph()
g.parse("http://www.w3.org/People/Berners-Lee/card")
v = g.serialize(format="xml")

AttributeError: 'str' object has no attribute 'rows'

In [11]:
g = Graph()
g.parse("graph.ttl")
newg = g.serialize(format = "n3")
newg


'@prefix ex: <http://www.example.com/> .\n@prefix mo: <http://purl.org/ontology/mo/> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix schema: <https://schema.org/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nex:0 a schema:MusicRecording ;\n    rdfs:label "Hey, Soul Sister"@en ;\n    schema:byArtist "http://www.example.com/Train" ;\n    schema:datePublished "2010"^^xsd:gYear ;\n    schema:genre "http://www.example.com/neo_mellow" .\n\nex:1 a schema:MusicRecording ;\n    rdfs:label "Love The Way You Lie"@en ;\n    schema:byArtist "http://www.example.com/Eminem" ;\n    schema:datePublished "2010"^^xsd:gYear ;\n    schema:genre "http://www.example.com/detroit_hip_hop" .\n\nex:10 a schema:MusicRecording ;\n    rdfs:label "Club Can\'t Handle Me (feat. David Guetta)"@en ;\n    schema:byArtist "http://www.example.com/Flo_Rida" ;\n    schema:datePublished "2010"^^xsd:gYear ;\n    schema:genre "http://www.example.com/dance_pop" .\n\nex:100 a schema:MusicRecording 

# 2. LIMES

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE LIMES SYSTEM "limes.dtd">

<LIMES>
    <PREFIX>
        <NAMESPACE>http://www.w3.org/1999/02/22-rdf-syntax-ns#</NAMESPACE>
        <LABEL>rdf</LABEL>
    </PREFIX>
    <PREFIX>
        <NAMESPACE>http://www.w3.org/2000/01/rdf-schema#</NAMESPACE>
        <LABEL>rdfs</LABEL>
    </PREFIX>
    <PREFIX>
        <NAMESPACE>https://schema.org/</NAMESPACE>
        <LABEL>schema</LABEL>
    </PREFIX>
    <PREFIX>
        <NAMESPACE>http://www.w3.org/2002/07/owl#</NAMESPACE>
        <LABEL>owl</LABEL>
    </PREFIX>
    <PREFIX>
        <NAMESPACE>http://www.w3.org/2001/XMLSchema#</NAMESPACE>
        <LABEL>xsd</LABEL>
    </PREFIX>

    <SOURCE>
        <ID>music</ID>
        <ENDPOINT>music.csv</ENDPOINT>
        <VAR>?song</VAR>
        <PAGESIZE>5000</PAGESIZE>
        <RESTRICTION>isLiteral(?song)</RESTRICTION>
        <PROPERTY>rdfs:label</PROPERTY>
        <TYPE>CSV</TYPE>
        <SEPARATOR>,</SEPARATOR>
        <ENCLOSURE>"</ENCLOSURE>
    </SOURCE>

    <TARGET>
        <ID>dbpedia</ID>
        <ENDPOINT>http://dbpedia.org/sparql</ENDPOINT>
        <VAR>?artist</VAR>
        <PAGESIZE>5000</PAGESIZE>
        <RESTRICTION>?artist a dbo:MusicalArtist</RESTRICTION>
        <PROPERTY>rdfs:label</PROPERTY>
        <TYPE>SPARQL</TYPE>
    </TARGET>

    <METRIC>levenshtein(x.rdfs:label, y.rdfs:label)</METRIC>

    <ACCEPTANCE>
        <THRESHOLD>0.8</THRESHOLD>
        <FILE>spotify_to_dbpedia_accepted.nt</FILE>
        <RELATION>owl:sameAs</RELATION>
    </ACCEPTANCE>

    <REVIEW>
        <THRESHOLD>0.6</THRESHOLD>
        <FILE>spotify_to_dbpedia_review.nt</FILE>
        <RELATION>owl:sameAs</RELATION>
    </REVIEW>

    <OUTPUT>NT</OUTPUT>
</LIMES>

#4. SPARQL

Q1. Return a list of artists and their names who produce songs with genres other than "pop" and "dance pop."

In [ ]:
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>
PREFIX mo: <http://purl.org/ontology/mo/>

SELECT DISTINCT ?artist ?name
WHERE {
  ?recording a schema:MusicRecording ;
             schema:byArtist ?artist ;
             schema:genre ?genre ;
             rdfs:label ?name .
  FILTER(?genre != ex:pop && ?genre != ex:dance_pop)
  ?artist a schema:Person .
}
ORDER BY ?name

Q2. Return a list of songs released in 2016 by artists born before 1990.

In [ ]:
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>
PREFIX mo: <http://purl.org/ontology/mo/>

SELECT ?song ?name ?year
WHERE {
  ?recording a schema:MusicRecording ;
             schema:datePublished ?year ;
             schema:byArtist ?artist ;
             rdfs:label ?name .
  FILTER(?year = "2016"^^xsd:gYear)
  ?artist a schema:Person ;
          schema:birthDate ?birth_date .
  FILTER(?birth_date < "1990-01-01"^^xsd:date)
}
ORDER BY ?name

Q3. Who is the artist that has produced the greatest number of songs?

In [ ]:
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>

SELECT ?artist (COUNT(?recording) AS ?count)
WHERE {
  ?recording a schema:MusicRecording ;
             schema:byArtist ?artist .
  ?artist a schema:Person .
}
GROUP BY ?artist
ORDER BY DESC(?count)
LIMIT 1

Q4. Return a list of artists born in the USA, sorted by the number of songs they have produced.


In [ ]:
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>

SELECT ?artist ?name (COUNT(?recording) AS ?count)
WHERE {
  ?recording a schema:MusicRecording ;
             schema:byArtist ?artist ;
             rdfs:label ?song .
  ?artist a schema:Person ;
          schema:birthPlace ?birth_place ;
          rdfs:label ?name .
  ?birth_place a schema:Place ;
               schema:addressCountry "USA" .
}
GROUP BY ?artist ?name
ORDER BY DESC(?count)

Q5. Find artists whose song names contain the word "love" and sort the artists by the resulting number of songs.

In [ ]:
PREFIX ex: <http://example.org/>
PREFIX schema: <https://schema.org/>

SELECT ?artist ?name (COUNT(?recording) AS ?count)
WHERE {
  ?recording a schema:MusicRecording ;
             schema:byArtist ?artist ;
             rdfs:label ?song .
  ?artist a schema:Person ;
          rdfs:label ?name .
  FILTER(CONTAINS(LCASE(?song), "love"))
}
GROUP BY ?artist ?name
ORDER BY DESC(?count)